In [1]:
import os
import numpy as np
import pandas as pd
import shutil

## Remove Top_Bott_Output If Exists Else Create Newly

In [2]:
if "Top_Bott_Output" in next(os.walk(os.getcwd()))[1]:
    shutil.rmtree("Top_Bott_Output")

os.mkdir("Top_Bott_Output")

## Reading Top & Bott Files

In [3]:
top = pd.read_csv("Top_Output/Top.csv", sep=';')
bott = pd.read_csv("Bott_Output/Bott.csv", sep=';')

## Adding Invoice, Venue_Code Columns

In [4]:
info = pd.read_excel("ntrcacentre_wr_15_oct13.xlsx")

In [5]:
top = pd.merge(top, info[["roll", "invoice", "exam_centre", "venue"]], left_on='Roll', right_on='roll', how='inner')

top = top.drop("roll", axis=1)
top = top.rename({"invoice": "Invoice", "exam_centre": "Exam_Centre", "venue": "Venue"}, axis=1)

## Renaming Litho

In [6]:
top = top.rename({"Litho_1": "Litho_1_Top", "Litho_2": "Litho_2_Top"}, axis=1)
bott = bott.rename({"Litho_1": "Litho_1_Bott", "Litho_2": "Litho_2_Bott"}, axis=1)

top["Litho_1_Top"] = top["Litho_1_Top"].astype(str)
top["Litho_2_Top"] = top["Litho_2_Top"].astype(str)

bott["Litho_1_Bott"] = bott["Litho_1_Bott"].astype(str)
bott["Litho_2_Bott"] = bott["Litho_2_Bott"].astype(str)

## Matching Litho_1_Top vs Litho_1_Bott

In [7]:
tdf = pd.merge(top, bott, left_on='Litho_1_Top', right_on='Litho_1_Bott', how='inner', suffixes=("_Top", "_Bott"))

df = tdf.copy()

tdf_litho = list(tdf["Litho_1_Top"])

top = top[~top["Litho_1_Top"].isin(tdf_litho)]
bott = bott[~bott["Litho_1_Bott"].isin(tdf_litho)]

## Matching Litho_1_Top vs Litho_2_Bott

In [8]:
tdf = pd.merge(top, bott, left_on='Litho_1_Top', right_on='Litho_2_Bott', how='inner', suffixes=("_Top", "_Bott"))

df = pd.concat([df, tdf])

tdf_litho = list(tdf["Litho_1_Top"])

top = top[~top["Litho_1_Top"].isin(tdf_litho)]
bott = bott[~bott["Litho_2_Bott"].isin(tdf_litho)]

## Matching Litho_2_Top vs Litho_1_Bott

In [9]:
tdf = pd.merge(top, bott, left_on='Litho_2_Top', right_on='Litho_1_Bott', how='inner', suffixes=("_Top", "_Bott"))

df = pd.concat([df, tdf])

tdf_litho = list(tdf["Litho_2_Top"])

top = top[~top["Litho_2_Top"].isin(tdf_litho)]
bott = bott[~bott["Litho_1_Bott"].isin(tdf_litho)]

## Matching Litho_2_Top vs Litho_2_Bott

In [10]:
tdf = pd.merge(top, bott, left_on='Litho_2_Top', right_on='Litho_2_Bott', how='inner', suffixes=("_Top", "_Bott"))

df = pd.concat([df, tdf])

tdf_litho = list(tdf["Litho_2_Top"])

top = top[~top["Litho_2_Top"].isin(tdf_litho)]
bott = bott[~bott["Litho_2_Bott"].isin(tdf_litho)]

## Seperating Pass and Fail Marks with Sorted Values

In [11]:
pass_marks = df[df["Written_Marks"] >= 40].sort_values(["Applicants_Subject_Code", "Written_Marks", "DoB(YMD)"], ascending=[1, 0, 1])
fail_marks = df[df["Written_Marks"] < 40]

df.to_csv("Top_Bott_Output/Tabulation.csv", index=False, sep=';')

pass_marks.to_csv("Top_Bott_Output/Pass.csv", index=False, sep=';')
fail_marks.to_csv("Top_Bott_Output/Fail.csv", index=False, sep=';')

top.to_csv("Top_Bott_Output/Top_Not_In_Bott.csv", index=False, sep=';')
bott.to_csv("Top_Bott_Output/Bott_Not_In_Top.csv", index=False, sep=';')